In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow import keras

from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import TextVectorization, StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cv2
import os

import matplotlib.image as mpimg

In [ ]:
def group_color(styles):
    styles["colorgroup"] = -1
    styles.loc[(styles.baseColour=='Red')|
           (styles.baseColour=='Brown')|
           (styles.baseColour=='Coffee Brown')|
           (styles.baseColour=='Maroon')|
           (styles.baseColour=='Rust')|
           (styles.baseColour=='Burgundy')|
           (styles.baseColour=='Mushroom Brown'),"colorgroup"] = 0
    styles.loc[(styles.baseColour=='Copper'),"colorgroup"] = 1
    styles.loc[(styles.baseColour=='Orange')|
               (styles.baseColour=='Bronze')|
               (styles.baseColour=='Skin')|
               (styles.baseColour=='Nude'),"colorgroup"] = 2
    styles.loc[(styles.baseColour=='Gold')|
               (styles.baseColour=='Khaki')|
               (styles.baseColour=='Beige')|
               (styles.baseColour=='Mustard')|
               (styles.baseColour=='Tan')|
               (styles.baseColour=='Metallic'),"colorgroup"]= 3
    styles.loc[(styles.baseColour=='Yellow'),"colorgroup"] = 4
    styles.loc[(styles.baseColour=='Lime Green'),"colorgroup"]= 5
    styles.loc[(styles.baseColour=='Green')|
           (styles.baseColour=='Sea Green')|
           (styles.baseColour=='Fluorescent Green')|
           (styles.baseColour=='Olive'),"colorgroup"] = 6
    styles.loc[(styles.baseColour=='Teal')|
           (styles.baseColour=='Turquoise Blue'),"colorgroup"] = 7
    styles.loc[(styles.baseColour=='Blue'),"colorgroup"]= 8
    styles.loc[(styles.baseColour=='Navy Blue'),"colorgroup"] = 9
    styles.loc[(styles.baseColour=='Purple')|
           (styles.baseColour=='Lavender'),"colorgroup"] = 10
    styles.loc[(styles.baseColour=='Pink')|
           (styles.baseColour=='Magenta')|
           (styles.baseColour=='Peach')|
           (styles.baseColour=='Rose')|
           (styles.baseColour=='Mauve'),"colorgroup"] = 11
    styles.loc[(styles.baseColour=='Black')|
           (styles.baseColour=='Charcoal'),"colorgroup"] = 12
    styles.loc[(styles.baseColour=='White')|
           (styles.baseColour=='Off White')|
           (styles.baseColour=='Cream'),"colorgroup"] = 13
    styles.loc[(styles.baseColour=='Grey')|
           (styles.baseColour=='Silver')|
           (styles.baseColour=='Taupe')|
           (styles.baseColour=='Grey Melange'),"colorgroup"] = 14
    styles.loc[(styles.baseColour=='Multi'),"colorgroup"] = 15

In [ ]:
def df_drop(styles, col, item):
    for i in item:
        styles = styles.drop(styles[styles[col] == i].index)
    return styles

In [ ]:
def get_df():
    
    styles = pd.read_csv("/kaggle/input/fashion-product-images-small/styles.csv", on_bad_lines='skip')
    styles = styles.drop(["productDisplayName"],axis = 1) #drop useless column, we do not need name to do recommendation
    styles = styles.drop(["year"],axis = 1) #drop useless column, we do not need year to do recommendation
    styles = styles[(styles.masterCategory=='Apparel')| (styles.masterCategory=='Footwear')] # drop useless rows, we are not recommend acessories
    styles = styles.drop(styles[styles["subCategory"] == "Innerwear"].index) # drop useless row, we are not recommend innerwears, only outfits.
    styles = styles.dropna() # drop NA
    styles = df_drop(styles,"subCategory", ["Apparel Set", "Dress","Loungewear and Nightwear","Saree","Socks"]) # we only recommend outfits.
    styles["subCategory"] = styles["subCategory"].transform(lambda x: "Footwear" if(x in ["Shoes","Flip Flops","Sandal"]) else x) # Group them into one category.
    styles = styles.drop(labels=[6695,16194,32309,36381,40000], axis=0) # drop incomplete rows
    group_color(styles) # group the color in to color-wheel
    return styles

In [ ]:
def make_input_array_subcate(df):
    train_images = np.zeros((len(df.id),80,60,3))
    for i in range(len(df.id)):
        
        #try:
        ID = df.id.iloc[i]
        path = f"/kaggle/input/fashion-product-images-small/images/{ID}.jpg"  
        img = cv2.imread(path)
        if img.shape != (80,60,3):
            img = image.load_img(path, target_size=(80,60,3))

        #except:
            #print(ID)
        
        train_images[i] = img
    
    data = tf.data.Dataset.from_tensor_slices(
      (
        {
          "images" : train_images
       },

        {
          "subCategory" : df[["subCategory"]]
        }
      )
    )

    return data

In [ ]:
def make_branch(res_input, n_out, act_type, name):
    z = layers.Dense(512, activation="relu")(res_input)
    z = layers.Dense(256, activation='relu')(z)
    z = layers.Dense(128, activation='relu')(z)
    z = layers.Dense(64, activation='relu')(z)

    z = layers.Dense(n_out)(z)
    z = layers.Activation(act_type, name=name)(z)
    return z

In [ ]:
def build_model(width, height):
    res50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(80,60,3))
    res50.trainable=False
    inputs = keras.Input(shape=(width,height,3),name = "images")
    x = res50(inputs, training=False)
    x = layers.Conv2D(32, (2, 2), activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    # -------------------------

    sub_branch = make_branch(x, len(le.classes_), 'softmax', 'subCategory')

    model = keras.Model(inputs=inputs,
                outputs=[sub_branch]
                       )
    return model

In [ ]:
def make_input_xx(x):#make_input_array_subcate(styles)
  x_input = x
  x_input = x_input.shuffle(buffer_size = len(x_input))

  x_train_size = int(0.6*len(x_input))
  x_val_size   = int(0.2*len(x_input))

  x_train = x_input.take(x_train_size).batch(2)
  x_val   = x_input.skip(x_train_size).take(x_val_size).batch(2)
  x_test  = x_input.skip(x_train_size + x_val_size).batch(2)

  return x_train,x_val,x_test

In [ ]:
def my_le(styles):
  articleTypeLB = LabelEncoder()
  genderLB = LabelEncoder()
  baseColourLB = LabelEncoder()
  seasonLB = LabelEncoder()
  usageLB = LabelEncoder()

  styles['articleType'] = articleTypeLB.fit_transform(styles['articleType'])
  styles['gender'] = genderLB.fit_transform(styles['gender'])
  styles['baseColour'] = baseColourLB.fit_transform(styles['baseColour'])
  styles['season'] = seasonLB.fit_transform(styles['season'])
  styles['usage'] = usageLB.fit_transform(styles['usage'])
  return styles,articleTypeLB,genderLB,baseColourLB,seasonLB,usageLB

In [ ]:
def get_234_df(x):
    styles = pd.read_csv("/kaggle/input/fashion-product-images-small/styles.csv", on_bad_lines='skip')
    styles = styles.drop(["productDisplayName"],axis = 1)
    styles = styles.drop(["year"],axis = 1)
    styles = styles[(styles.masterCategory=='Apparel')| (styles.masterCategory=='Footwear')]
    styles = styles.drop(styles[styles["subCategory"] == "Innerwear"].index)
    styles = styles.dropna()
    styles = df_drop(styles,"subCategory", ["Apparel Set", "Dress","Loungewear and Nightwear","Saree","Socks"])
    styles["subCategory"] = styles["subCategory"].transform(lambda x: "Footwear" if(x in ["Shoes","Flip Flops","Sandal"]) else x)
    styles = styles.drop(labels=[6695,16194,32309,36381,40000], axis=0)
    styles = styles[styles.subCategory == x]
    group_color(styles)
    styles.baseColour=styles.colorgroup

    return styles

In [ ]:
#

In [ ]:
def make_input_array_2(df):
    valid_ids = []
    train_images = []

    for i in range(len(df.id)):
        ID = df.id.iloc[i]
        path = f"/kaggle/input/fashion-product-images-small/images/{ID}.jpg"
        
        # Check if file exists
        if not os.path.exists(path):
            print(f"File does not exist: {path}")
            continue
        
        img = cv2.imread(path)
        
        # Check if image is loaded successfully
        if img is None:
            print(f"Warning: Could not load image {ID} at {path}")
            continue
        
        # Resize if necessary
        if img.shape != (80, 60, 3):
            img = image.load_img(path, target_size=(80, 60, 3))
            img = image.img_to_array(img)
        
        train_images.append(img / 255.0)
        valid_ids.append(ID)
    
    # Update the DataFrame to include only valid IDs
    df_valid = df[df.id.isin(valid_ids)]
    
    # Convert images and corresponding data to a TensorFlow dataset
    data = tf.data.Dataset.from_tensor_slices(
        (
            {"images": np.array(train_images)},
            {
                "articleType": df_valid[["articleType"]].values,
                "gender": df_valid[["gender"]].values,
                "baseColour": df_valid[["baseColour"]].values,
                "season": df_valid[["season"]].values,
                "usage": df_valid[["usage"]].values
            }
        )
    )

    return data

In [ ]:
styles = get_df()
styles["subCategory"].unique() # we can check by this code that we only have three subcategory now.

"""# Model-1: """
le = LabelEncoder()
styles["subCategory"] = le.fit_transform(styles["subCategory"])
styles.head()
le.classes_
sub_train,sub_val,sub_test = make_input_xx(make_input_array_subcate(styles))
sub_model = build_model(80, 60)
sub_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(sub_model)

sub_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

sub_history = sub_model.fit(sub_train, 
                    epochs=5, 
                    steps_per_epoch = 2000,
                    validation_data = sub_val)

In [ ]:
sub_model.save("model_sub.h5")

In [ ]:
test_model = tf.keras.models.load_model("model_sub.h5")
test_model.evaluate(sub_test)

In [ ]:
def build_model(width, height, articleTypeLB,genderLB,baseColourLB,seasonLB,usageLB):
    # -------------------------
    res50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(80,60,3))
    res50.trainable=False
    inputs = keras.Input(shape=(width,height,3),name = "images")
    x = res50(inputs, training=False)
    
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    # -------------------------

    article_branch = make_branch(x, len(articleTypeLB.classes_), 'softmax', 'articleType')
    gender_branch = make_branch(x, len(genderLB.classes_), 'softmax', 'gender')
    color_branch = make_branch(x, len(baseColourLB.classes_), 'softmax', 'baseColour')
    season_branch = make_branch(x, len(seasonLB.classes_), 'softmax', 'season')
    usage_branch = make_branch(x, len(usageLB.classes_), 'softmax', 'usage')

    model = keras.Model(inputs=inputs,
                outputs=[article_branch, gender_branch, color_branch, 
                            season_branch, usage_branch]
                       )
    return model

In [ ]:
"""# Model 234"""
top_df = get_234_df("Topwear")
bottom_df = get_234_df("Bottomwear")
foot_df = get_234_df("Footwear")

top_df,top_art,top_gen,top_base,top_sea,top_usage = my_le(top_df)
bottom_df,bottom_art,bottom_gen,bottom_base,bottom_sea,bottom_usage = my_le(bottom_df)
foot_df,foot_art,foot_gen,foot_base,foot_sea,foot_usage = my_le(foot_df)

foot_usage.classes_

top_base_model = build_model(80,60,top_art,top_gen,top_base,top_sea,top_usage)
bottom_base_model = build_model(80,60,bottom_art,bottom_gen,bottom_base,bottom_sea,bottom_usage)
foot_base_model = build_model(80,60,foot_art,foot_gen,foot_base,foot_sea,foot_usage)


top_train, top_val, top_test = make_input_xx(make_input_array_2(top_df))
bottom_train, bottom_val, bottom_test = make_input_xx(make_input_array_2(bottom_df))
foot_train, foot_val, foot_test = make_input_xx(make_input_array_2(foot_df))

top_base_model.compile(
    optimizer='adam',
    loss={
        'articleType': 'sparse_categorical_crossentropy',
        'gender': 'sparse_categorical_crossentropy',
        'baseColour': 'sparse_categorical_crossentropy',
        'season': 'sparse_categorical_crossentropy',
        'usage': 'sparse_categorical_crossentropy'
    },
    metrics={
        'articleType': 'accuracy',
        'gender': 'accuracy',
        'baseColour': 'accuracy',
        'season': 'accuracy',
        'usage': 'accuracy'
    }
)
bottom_base_model.compile(
    optimizer='adam',
    loss={
        'articleType': 'sparse_categorical_crossentropy',
        'gender': 'sparse_categorical_crossentropy',
        'baseColour': 'sparse_categorical_crossentropy',
        'season': 'sparse_categorical_crossentropy',
        'usage': 'sparse_categorical_crossentropy'
    },
    metrics={
        'articleType': 'accuracy',
        'gender': 'accuracy',
        'baseColour': 'accuracy',
        'season': 'accuracy',
        'usage': 'accuracy'
    }
)
foot_base_model.compile(
    optimizer='adam',
    loss={
        'articleType': 'sparse_categorical_crossentropy',
        'gender': 'sparse_categorical_crossentropy',
        'baseColour': 'sparse_categorical_crossentropy',
        'season': 'sparse_categorical_crossentropy',
        'usage': 'sparse_categorical_crossentropy'
    },
    metrics={
        'articleType': 'accuracy',
        'gender': 'accuracy',
        'baseColour': 'accuracy',
        'season': 'accuracy',
        'usage': 'accuracy'
    }
)

In [ ]:
top_history = top_base_model.fit(top_train, 
                    epochs=10, 
                    steps_per_epoch = 500,
                    validation_data = top_val)

top_base_model.evaluate(top_test)
top_base_model.save("model_2.1.h5")

In [ ]:
bottom_history = bottom_base_model.fit(bottom_train, 
                    epochs=15, 
                    steps_per_epoch = 50,
                    validation_data = bottom_val)

bottom_base_model.evaluate(bottom_test)

bottom_base_model.save("model_2.2.h5")

In [ ]:
foot_history = foot_base_model.fit(foot_train, 
                    epochs=5, 
                    steps_per_epoch = 2000,
                    validation_data = foot_val)

foot_base_model.evaluate(foot_test)

foot_base_model.save("model_2.2.h5")